In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession, functions as F
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/06 13:03:27 WARN Utils: Your hostname, USERs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.11.129 instead (on interface en0)
23/03/06 13:03:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 13:03:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/06 13:03:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2023-03-06 11:35:30--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.187, 18.155.128.6, 18.155.128.222, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet.1’

fhvhv_tripdata_2021 100%[===================>] 294.61M  1.18MB/s    in 1m 57s  

2023-03-06 11:37:28 (2.51 MB/s) - ‘fhvhv_tripdata_2021-01.parquet.1’ saved [308924937/308924937]



In [3]:
!wc -l fhvhv_tripdata_2021-01.parquet

 1006794 fhvhv_tripdata_2021-01.parquet


In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [5]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [6]:
!head -n 101 fhvhv_tripdata_2021-01.parquet > head.parquet

In [7]:
df_pandas = pd.read_parquet('fhvhv_tripdata_2021-01.parquet')

In [8]:
df_pandas.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [9]:
df_conv = df_pandas[:102]

In [10]:
spark.createDataFrame(df_conv).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [11]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [27]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.parquet')

In [28]:
df = df.repartition(24)

In [30]:
df.write.parquet('fhvhv/2021/01/')

23/03/06 13:14:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///Users/user/Desktop/Data-Engineering/DE-project1/Data-Engineering-Project/week-5/code/fhvhv_tripdata_2021-01.parquet


In [39]:
df = spark.read.parquet('fhvhv/2021/01/')

In [40]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



### SELECT * FROM df WHERE hvfhs_license_num = HV0003

In [41]:
df.show()

+---------------------+--------------------+---------------+----------------+------------+------------+-------+
|    hvfhs_license_num|dispatching_base_num|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+---------------------+--------------------+---------------+----------------+------------+------------+-------+
| ]���G�\v�߽k1��...|���F�{��|�<y...|           null|            null|        null|        null|   null|
| zS����-s�-^z#F...|                null|           null|            null|        null|        null|   null|
| uD�����j>��e4?\f...|                null|           null|            null|        null|        null|   null|
| ��de�b�������...|                null|           null|            null|        null|        null|   null|
| ��%}N�!�g�/VGߏ�...|                null|           null|            null|        null|        null|   null|
| �BmX\a!���#%�z��...|                null|           null|            null|        null|        null| 

In [42]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [43]:
crazy_stuff('B02884')

's/b44'

In [44]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [49]:
df \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'hvfhs_license_num') \
    .show()

+-------+-------------------+-------------------+------------+------------+-----------------+
|base_id|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|hvfhs_license_num|
+-------+-------------------+-------------------+------------+------------+-----------------+
|  a/a7a|2021-01-01 01:33:44|2021-01-01 01:49:07|         230|         166|           HV0003|
|  a/a7a|2021-01-01 01:55:19|2021-01-01 02:18:21|         152|         167|           HV0003|
|  e/acc|2021-01-01 01:23:56|2021-01-01 01:38:05|         233|         142|           HV0003|
|  e/acc|2021-01-01 01:42:51|2021-01-01 01:45:50|         142|         143|           HV0003|
|  e/acc|2021-01-01 01:48:14|2021-01-01 02:08:42|         143|          78|           HV0003|
|  e/9ce|2021-01-01 01:06:59|2021-01-01 01:43:01|          88|          42|           HV0005|
|  e/9ce|2021-01-01 01:50:00|2021-01-01 02:04:57|          42|         151|           HV0005|
|  e/acc|2021-01-01 01:14:30|2021-01-01 01:50:27|          7

In [50]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint]

23/03/07 05:59:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 6393310 ms exceeds timeout 120000 ms
23/03/07 05:59:37 WARN SparkContext: Killing executors is not supported by current scheduler.
23/03/07 05:59:38 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at s

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49717)
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/opt/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/opt/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 257, in accum_updates
  